## Sliding 3D

In [5]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
import pylops

from pylops.signalprocessing.sliding3d import sliding3d_design
from pylops.utils.describe import describe

from pylops.signalprocessing import Sliding3D
from sliding3dold import Sliding3D as Sliding3DOLD

USE_CUPY = True

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
if USE_CUPY:
    import cupy as np
    from cupyx.profiler import benchmark
    np_asarray = np.asarray
    np_asnumpy = np.asnumpy
    np_float = np.float32
    np_floatc = np.complex64
    mempool = np.get_default_memory_pool()
    fftengine = 'numpy'
    fftkwargs = dict()
else:
    np_asarray = np.asarray
    np_asnumpy = np.asarray
    np_float = np.float64
    np_floatc = np.complex128
    fftengine = 'scipy'
    fftkwargs = dict(workers=16)

In [7]:
def bench_Op(Op, x):
    return Op @ x

def bench_OpH(Op, x):
    return Op.H @ x

In [8]:
nwin = (26, 24)
nover = (11, 9)
nop = 32
#dimsd = (200, 200, 32) # small
dimsd = (400, 400, 32) # large
tapertype = None # 'cosine' # cannot match old for other than None, as edges were not implemented...

y = np.arange(dimsd[0]*dimsd[1]*dimsd[2]).reshape(dimsd).astype(np_float)

nwins, dims, _, _ = sliding3d_design(dimsd, nwin, nover, (nop, nop, (nop + 2) // 2))

# no operator broadcast
Op = pylops.signalprocessing.FFTND((nwin[0], nwin[1], dimsd[2]), nffts=(nop, nop, nop),
                                   axes=(-3, -2, -1), sampling=(1, 1, 1), 
                                   real=True, engine=fftengine,
                                   dtype=np_floatc, **fftkwargs)
Slid = Sliding3DOLD(
    Op.H, dims, dimsd, nwin, nover, (nop, nop, (nop + 2) // 2), tapertype=tapertype)
Slid1a = Sliding3D(Op.H, dims, dimsd, nwin, nover, (nop, (nop + 2) // 2), tapertype=tapertype)

# with operator broadcast
Op = pylops.signalprocessing.FFTND((nwins[0], nwins[1], nwin[0], nwin[1], dimsd[2]), nffts=(nop, nop, nop), 
                                   axes=(-3, -2, -1), sampling=(1, 1, 1), real=True, 
                                   engine=fftengine,
                                   dtype=np_floatc, **fftkwargs)
Slid1b = Sliding3D(Op.H, dims, dimsd, nwin, nover, (nop, nop, (nop + 2) // 2), tapertype=tapertype)

x = Slid.H * y.ravel()

 270 285 300 315 330 345 360], end:[ 26  41  56  71  86 101 116 131 146 161 176 191 206 221 236 251 266 281
 296 311 326 341 356 371 386] / start:[  0  15  30  45  60  75  90 105 120 135 150 165 180 195 210 225 240 255
 270 285 300 315 330 345 360 375], end:[ 24  39  54  69  84  99 114 129 144 159 174 189 204 219 234 249 264 279
 294 309 324 339 354 369 384 399]
 576 608 640 672 704 736 768], end:[ 32  64  96 128 160 192 224 256 288 320 352 384 416 448 480 512 544 576
 608 640 672 704 736 768 800] / start:[  0  32  64  96 128 160 192 224 256 288 320 352 384 416 448 480 512 544
 576 608 640 672 704 736 768 800], end:[ 32  64  96 128 160 192 224 256 288 320 352 384 416 448 480 512 544 576
 608 640 672 704 736 768 800 832]


In [9]:
print(np.allclose(Slid @ x, Slid1a @ x), np.allclose(Slid.H @ y, Slid1a.H @ y))
print(np.allclose(Slid @ x, Slid1b @ x), np.allclose(Slid.H @ y, Slid1b.H @ y))

True True
True True


In [10]:
if not USE_CUPY:
    %timeit -n 5 -r 10 Slid * x # OLD
    %timeit -n 5 -r 10 Slid1a * x # NEW
    %timeit -n 5 -r 10 Slid1b * x # NEW with Op broadcasted
else:
    print(benchmark(bench_Op, (Slid, x,), n_repeat=20))
    print(benchmark(bench_Op, (Slid1a, x,), n_repeat=20))
    print(benchmark(bench_Op, (Slid1b, x,), n_repeat=20))

bench_Op            :    CPU: 334910.708 us   +/- 2520.416 (min: 330315.383 / max: 340415.027) us     GPU-0: 338737.880 us   +/- 2491.911 (min: 334217.377 / max: 344182.770) us
bench_Op            :    CPU: 303479.441 us   +/- 2227.488 (min: 299053.564 / max: 308423.038) us     GPU-0: 303485.713 us   +/- 2228.447 (min: 299058.167 / max: 308430.603) us
bench_Op            :    CPU: 14044.057 us   +/- 508.498 (min: 13504.756 / max: 15811.414) us     GPU-0: 14050.869 us   +/- 508.810 (min: 13510.688 / max: 15818.752) us


In [11]:
if not USE_CUPY:
    %timeit -n 5 -r 10 Slid.H * y # OLD
    %timeit -n 5 -r 10 Slid1a.H * y # NEW
    %timeit -n 5 -r 10 Slid1b.H * y # NEW with Op broadcasted
else:
    print(benchmark(bench_OpH, (Slid, y,), n_repeat=20))
    print(benchmark(bench_OpH, (Slid1a, y,), n_repeat=20))
    print(benchmark(bench_OpH, (Slid1b, y,), n_repeat=20))

bench_OpH           :    CPU: 216526.531 us   +/- 1794.010 (min: 214167.524 / max: 220645.720) us     GPU-0: 216533.792 us   +/- 1795.673 (min: 214174.438 / max: 220652.283) us
bench_OpH           :    CPU: 188689.008 us   +/- 1378.948 (min: 185586.305 / max: 190813.516) us     GPU-0: 188695.734 us   +/- 1378.460 (min: 185600.067 / max: 190820.160) us
bench_OpH           :    CPU:   665.198 us   +/- 19.420 (min:   631.155 / max:   708.099) us     GPU-0:  2263.472 us   +/- 13.848 (min:  2240.512 / max:  2310.144) us


In [12]:
# Checking sliding with taper
nwin = (26, 24)
nover = (11, 9)
nop = 32
dimsd = (191, 189, 32) # small
tapertype = 'cosine'

nwins, dims, _, _ = sliding3d_design(dimsd, nwin, nover, (nwin[0], nwin[1], dimsd[2]))

In [13]:
# no operator broadcast
Op = pylops.Identity((nwin[0], nwin[1], dimsd[2]))
Slid1a = Sliding3D(Op.H, dims, dimsd, nwin, nover, (nwin[1], dimsd[2]), 
                   tapertype=tapertype)

x = np.ones(Slid1a.dims) + 0.
y = Slid1a @ x

np.unique(y)

array([1., 1.])

In [14]:
# no operator broadcast
Op = pylops.Identity((nwins[0], nwins[1], nwin[0], nwin[1], dimsd[2]))
Slid1b = Sliding3D(Op.H, dims, dimsd, nwin, nover, (nwin[1], dimsd[2]), 
                   tapertype=tapertype)

x = np.ones(Slid1a.dims) + 0.
y = Slid1a @ x

np.unique(y)

array([1., 1.])